# Install Python Packages

In [4]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith

In [5]:
%pip install --quiet -U pydantic

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Check Environment Dependencies

In [7]:
import sys
print(sys.executable)
#!pip list

/opt/conda/bin/python


## Setup logging

In [8]:
import logging

logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)


# Setup OpenAI

In [6]:
from openai_config import OpenAIConfig

In [44]:
# We read the .env file inside the class
openai_config = OpenAIConfig()
openai_config.chat_model.invoke("Tell me a Joke about corporate life")

2024-08-29 08:21:20,946 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Why did the employee bring a ladder to work?\n\nBecause they heard the job was a "climb the corporate ladder" kind of position!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 14, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, id='run-1b673d15-a85c-4d79-9629-09393927d015-0', usage_metadata={'input_tokens': 14, 'output_tokens': 28, 'total_tokens': 42})

In [8]:
import os
logger.debug(os.getenv('OPENAI_API_KEY'))
logger.info(os.getenv('LANGCHAIN_PROJECT'))

2024-08-28 23:57:30,518 - INFO - builder


# High Level Plan

In [107]:
high_level_query = """You are a DevSecOs expert. You are thorough and leave nothing to chance. 

Create an end to end plan to debug Kubernetes DevOps Incidents on a  
environment with multiple clusters and namespaces. The plan must follows established 
DevSecOps best practices.  

The plan should be generic and not point to any specific issue.

The incident starts with a notification in a Slack room with the PagerDuty incident ID

Use the following methodology to create the plan. These steps can be repeated any number of times. 

strings between brackets are placeholders that you must fill in as you create the tasks and plan: 


Thought: To solve this problem, I need to break it down into smaller, more manageable steps. I will think through the key components of the task.
Action: Identify the main goals and subgoals required to complete the task.
Observation: The key goals are [list high-level goals]. To achieve these, I will need to complete the following tasks: [list tasks].
Thought: Now that I have a plan and a list of the task, I will start working through the steps systematically.
Action: Begin executing the first task by [describe first action].
Observation: Completing the first task revealed that [describe observation]. I will need to adjust my approach slightly.
Thought: Based on the results so far, I should modify my plan to be more effective. I will re-evaluate the next steps.
Action: Revise the plan by [describe revised action].
Observation: The revised approach is working better because [describe positive outcome]. I'm making good progress towards the overall goal.
Thought: I believe I now have enough information to provide a final solution to the original problem.
Action: Summarize the key steps taken and provide the final solution.
Observation: The solution is: [provide final solution with all necessary details for each step].

"""

In [108]:
high_level_response = openai_config.chat_model.invoke(high_level_query)

2024-08-28 01:24:12,233 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [109]:
json_string = high_level_response.content.strip('```json\n').strip('```')

In [110]:
print(json_string)

### End-to-End Plan for Debugging Kubernetes DevOps Incidents

---

**Thought:** To solve this problem, I need to break it down into smaller, more manageable steps. I will think through the key components of the task.

---

**Action:** Identify the main goals and subgoals required to complete the task.

**Main Goals:**
1. Acknowledge the incident and gather information.
2. Analyze the incident across clusters and namespaces.
3. Identify the root cause of the incident.
4. Implement a resolution and validate it.
5. Document the incident and update processes.

**Subgoals:**
- Collect logs and metrics.
- Communicate with relevant teams.
- Review configurations and deployments.
- Implement a fix.
- Conduct post-mortem and update incident response documentation.

---

**Thought:** Now that I have a plan for tackling the task, I will start working through the steps systematically.

---

**Action:** Begin executing the first subtask by acknowledging the incident and gathering information from 

# Structured Output Plan

## References

https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI.with_structured_output

## Define Schemas

### Tool Schema

In [9]:
from pydantic import BaseModel, Field
from typing import List, Union

class Tool(BaseModel):
    """
    The `Tool` class represents a tool used in DevOps tasks. Each tool should include relevant details that define how it is used within a task.

    Key Points:
    - **Name**: The name of the tool as known in the DevOps industry.
    - **Description**: A clear, human-readable description of the tool, outlining its purpose and functionality.
    - **Command**: A function call or a complete Unix command with relevant options should always be provided. This is crucial for tools that access on-prem resources or interact with external services via REST/cURL. For example, 'kubectl get pods --namespace default' should be provided for `kubectl`, even if it interacts with external resources.
    - **URL**: An optional URL where more information about the tool can be found. This is useful for users who need additional details or documentation.

    **Examples of Proper Usage:**
    - **`kubectl`**: Command might be 'kubectl get pods --namespace default'.
    - **`curl`**: Command might be 'curl -X GET https://api.example.com/resource'.

    **Examples of What Should Not Be Included:**
    - Omitting the command for a tool (e.g., only providing 'kubectl' without any command).
    - Providing incomplete or vague commands that do not specify how to use the tool (e.g., 'curl' without specifying the URL or method).

    Attributes:
    - `name` (str): The name known in the DevOps industry, e.g., 'kubectl'.
    - `description` (str): A human-like description of the tool, e.g., 'A command-line tool for controlling Kubernetes clusters.'
    - `command` (Union[str, None]): A function call or a complete Unix command with relevant options should always be provided. This is crucial for tools that access on-prem resources or interact with external services via REST/cURL. For example, 'kubectl get pods --namespace default' should be provided for `kubectl`, even if it interacts with external resources.
    - `url` (str, Optional): URL for more information about the tool, e.g., 'https://kubernetes.io/docs/reference/kubectl/'. Optional but recommended for additional context.

    """

    name: str = Field(..., description="The name known in the DevOps industry, e.g., 'kubectl'")
    description: str = Field(..., description="A human-like description of the tool, e.g., 'A command-line tool for controlling Kubernetes clusters.'")
    command: Union[str, None] = Field(None, description="A function call or a complete Unix command with relevant options should always be provided. This is crucial for tools that access on-prem resources or interact with external services via REST/cURL. For example, 'kubectl get pods --namespace default' should be provided for `kubectl`, even if it interacts with external resources.")
    url: str = Field(None, description="URL for more information about the tool, e.g., 'https://kubernetes.io/docs/reference/kubectl/'")

    class Config:
        """Configuration for the Tool model."""
        json_schema_extra = {
            "description": "The `Tool` class represents a tool used in DevOps tasks."
            # Removed 'examples' as it is now handled at the field level
        }

### Task Schema

In [29]:
class Task(BaseModel):
    """
    The `Task` class represents an atomic unit of work within a plan. Each task must be focused on a single, clear objective and should not encompass multiple, unrelated activities.

    Key Points:
    - A task should perform one specific function and should not combine different activities. 
    - Each task should have a clear, verifiable success criterion that is achievable through the task's activities alone.
    - Tasks must be URL-friendly, without spaces or special characters in their names.
    
    **Examples of What a Task Should NOT Do:**
    - "Acknowledge the incident in PagerDuty and notify the team in the Slack room." (This task mixes incident management with team communication.)
    - "Update the application code and deploy it to production." (This task combines code changes with deployment, which should be separated.)
    - "Collect all relevant information about the incident from Slack and PagerDuty" (This task has two action items)

    Attributes:
    - `name` (str): URL-friendly name for the task, e.g., 'deploy-application'. Must not contain spaces or special characters.
    - `description` (str): A detailed description of the task, e.g., 'Deploy the application to the Kubernetes cluster using Helm.'
    - `success_criteria` (str): Clear and verifiable criteria for task completion, e.g., 'Application is successfully deployed and running in the Kubernetes cluster, as verified by checking the deployment status with 'kubectl get deployments'.' Each task must be atomic and focus on one thing only.
    - `tools` (List[Tool]): List of `Tool` instances required for the task, specifying which tools are needed.

    """    
    name: str = Field(..., description="URL-friendly name for the task, e.g., 'deploy-application'. Must not contain spaces or special characters.")
    description: str = Field(..., description="A detailed description of the task, e.g., 'Deploy the application to the Kubernetes cluster using Helm.'")
    success_criteria: str = Field(..., description="Clear and verifiable criteria for task completion, e.g., 'Application is successfully deployed and running in the Kubernetes cluster, as verified by checking the deployment status with 'kubectl get deployments'.' Each task must be atomic and focus on one thing only.")
    tools: List[Tool] = Field(..., description="List of `Tool` instances required for the task, specifying which tools are needed.")

    class Config:
        """Configuration for the Task model."""
        json_schema_extra = {
            "description": "The `Task` class represents an atomic unit of work that involves using specific tools to achieve a clear goal."
            # Removed 'examples' as it is now handled at the field level
        }

### Plan Schema

In [30]:
from langchain_core.messages import BaseMessage


class Plan(BaseModel):
    tasks: List['Task'] = Field(
        ..., 
        description=(
            "A list of `Task` instances that constitute the plan. Each task must be atomic, "
            "focused on a specific goal, and the entire plan must be complete, verifiable, "
            "and reproducible. It should follow DevOps best practices to resolve incidents " 
            "to ensure that the plan is executed consistently and reliably across different "
            "environments."
        )
    )
    Reasoning: str = Field(
        ..., 
        description=(
            "OpenAI should capture here the entire reasoning sequence used to create "
            " the plan"
        )
    )


## Invoke LLM

This forces the LLM to use JSON response based on the given schema

In [31]:
structured_system_query = """You are a DevSecOs expert. You are thorough and leave nothing to chance.  
"""

In [32]:
structured_user_query = """
Create an end to end plan to debug Kubernetes DevOps Incidents on a  
environment with multiple clusters and namespaces. The plan must follows established 
DevSecOps best practices.  

The incident starts with a notification in a Slack room with the PagerDuty incident ID 
"""

In [33]:
from langchain_core.prompts import ChatPromptTemplate

structured_template = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("user", "{input}")
])
structured_prompt = structured_template.invoke({"system_prompt": structured_system_query, "input": structured_user_query})

In [34]:
structured_llm = openai_config.chat_model.with_structured_output(Plan, method="json_schema")

In [35]:
structured_plan_response = structured_llm.invoke(structured_prompt)

2024-08-29 00:03:18,352 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [36]:
import json
print(json.dumps(structured_plan_response.dict(), indent=4))

{
    "tasks": [
        {
            "name": "acknowledge-incident",
            "description": "Acknowledge the incident in PagerDuty to prevent further notifications and inform the team that the issue is being addressed.",
            "success_criteria": "Incident is acknowledged in PagerDuty, and team members are notified in Slack that the incident is being addressed.",
            "tools": [
                {
                    "name": "PagerDuty",
                    "description": "An incident management tool that helps teams respond to incidents quickly.",
                    "command": "curl -X POST 'https://api.pagerduty.com/incidents/{incident_id}/acknowledge' -H 'Authorization: Token token={api_token}' -H 'Content-Type: application/json' -d '{\"incident\": {\"type\": \"incident_reference\"}}'",
                    "url": "https://support.pagerduty.com/docs/api-docs"
                }
            ]
        },
        {
            "name": "check-incident-details",
        

# Tool Calling

## Define Toolkit

In [20]:
# https://python.langchain.com/v0.2/docs/concepts/#toolkits
# https://python.langchain.com/v0.2/docs/how_to/custom_tools/
from incident import Incident
incident = Incident()
# Create a toolkit with the Incident class
incident_toolkit = incident.get_tools()

In [21]:
for tool in incident_toolkit:
    # Let's inspect some of the attributes associated with the tool.
    logger.debug(tool.name)
    logger.debug(tool.description)
    logger.debug(tool.args)
    logger.debug(tool.return_direct)

## Tool binding

In [22]:
import json
chat_model_tools = openai_config.chat_model.bind_tools(incident_toolkit)
# chat_model_tools.kwargs.get("tools")[0].get("function")["strict"] = True
# chat_model_tools.kwargs.get("tools")[0].get("function").get("parameters")["additionalProperties"] = False
# chat_model_tools.kwargs.get("tools")[1].get("function")["strict"] = True
# chat_model_tools.kwargs.get("tools")[1].get("function").get("parameters")["additionalProperties"] = False
# chat_model_tools.kwargs.get("tools")[2].get("function")["strict"] = True
# chat_model_tools.kwargs.get("tools")[2].get("function").get("parameters")["additionalProperties"] = False
# chat_model_tools.kwargs.get("tools")[0].get("function").get("strict")
# chat_model_tools.kwargs.get("tools")[1].get("function")
# chat_model_tools.kwargs.get("tools")[2].get("function")
# json_llm = chat_model_tools.bind(response_format={"type": "json_object"})
# logging.info(json.dumps(chat_model_tools.kwargs.get("tools")[0].get("function"), indent=4))

In [41]:
import json

tool_calling_query = f"""Can you help Acknowledge the incident 123 in PagerDuty to prevent further 
notifications and inform the team that the issue is being addressed \n"""

## Invoke LLM

In [42]:
tool_response = chat_model_tools.invoke(tool_calling_query)

2024-08-29 00:08:53,419 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [43]:
print(json.dumps(tool_response.tool_calls, indent=4))

[
    {
        "name": "read_incident",
        "args": {
            "incident_id": "123"
        },
        "id": "call_Fx451JxbF7jW3lhNHtxcrDUL",
        "type": "tool_call"
    }
]


# Tool Calling and Structured Output

In [179]:
structured_tools_system_query = """You are a DevSecOs expert. You are thorough and leave nothing to chance.  
"""

In [180]:
structured_tools_user_query = """
Create an end to end plan to debug Kubernetes DevOps Incidents on a  
environment with multiple clusters and namespaces. The plan must follows established 
DevSecOps best practices.

Always check if any tool specified in the message can be used in a task. 

The incident starts with a notification in a Slack room with the PagerDuty incident ID 
"""

In [205]:
from langchain_core.prompts import ChatPromptTemplate

structured_template = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("user", "{input}")
])
structured_tools_prompt = structured_template.invoke({"system_prompt": structured_tools_system_query, "input": structured_tools_user_query})

In [215]:
# lm.bind_tools([WeatherTool], response_format=Plan, strict=True)

chat_model_tools = openai_config.chat_model.bind_tools(incident_toolkit)
# structured_llm_tools = chat_model_tools.with_structured_output(Plan, method="json_schema", include_raw=True)

In [219]:
structured_llm_tools = chat_model_tools.with_structured_output(Plan, method="json_schema")

In [220]:
structured_plan_and_tools_response = structured_llm_tools.invoke(structured_tools_prompt)

2024-08-28 13:34:40,173 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [221]:
print(json.dumps(structured_plan_and_tools_response.dict(), indent=4))

{
    "tasks": [
        {
            "name": "initialize-incident-response",
            "description": "Initiate the incident response process by gathering necessary information about the incident from Slack and PagerDuty.",
            "success_criteria": "Incident response team is assembled, and all relevant information regarding the incident is collected from Slack and PagerDuty, including the incident ID, timestamps, and any reported issues.",
            "tools": [
                {
                    "name": "Slack",
                    "description": "Slack is a messaging platform used for team collaboration.",
                    "command": null,
                    "url": "https://slack.com/"
                },
                {
                    "name": "PagerDuty",
                    "description": "A tool for incident management and alerting.",
                    "command": null,
                    "url": "https://www.pagerduty.com/"
                }
            ]